In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import tasks.run_helper

In [2]:
df = pd.read_json("../data/toxicity_ratings.json", lines=True)
df = df.explode(column="ratings")

ratings_df = pd.json_normalize(df.ratings)
df = pd.concat([df.reset_index(), ratings_df.reset_index()], axis=1)
df = df.drop(columns=["ratings", "index"])

df = df.loc[
    :,
    [
        "comment",
        "toxic_score",
        "personally_seen_toxic_content",
        "personally_been_target",
        "identify_as_transgender",
        "toxic_comments_problem",
        "education",
        "age_range",
        "lgbtq_status",
        "political_affilation",
        "is_parent",
        "religion_important",
    ],
]
df = df.groupby("comment").agg(list)
df["random"] = tasks.preprocessing.get_rand_col(df, "education")
df = df.head(5000)

In [3]:
sdb_columns = [
    "personally_seen_toxic_content",
    "personally_been_target",
    "identify_as_transgender",
    "toxic_comments_problem",
    "education",
    "age_range",
    "lgbtq_status",
    "political_affilation",
    "is_parent",
    "religion_important",
]

res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    value_col="toxic_score",
    comment_key_col="comment",
)
res

Evaluating SDB dimensions:   0%|          | 0/10 [00:00<?, ?it/s]

0  \
                                                                                     kappa   
sdb_column                                                                                   
age_range                     18 - 24                                             0.080025   
                              25 - 34                                            -0.045394   
                              35 - 44                                            -0.022743   
                              45 - 54                                            -0.012402   
                              55 - 64                                            -0.000022   
                              65 or older                                        -0.070458   
                              Prefer not to say                                   0.000000   
                              Under 18                                            0.000000   
education                     Associate degree in college (2-year)                0.090353   
                              Bachelor's degree in college (4-year)              -0.039050   
                              Doctoral degree                                    -1.249190   
                              High school graduate (high school diploma or eq...  0.028990   
                              Less than high school degree                       -0.676737   
                              Master's degree                                    -0.032379   
                              Other                                               0.000000   
                              Prefer not to say                                   0.000000   
                              Professional degree (JD, MD)                       -0.099932   
                              Some college but no degree                          0.011888   
                              NaN                                                 0.000000   
identify_as_transgender       No                                                 -0.006181   
                              Prefer not to say                                  -0.072016   
                              Yes                                                -0.063646   
is_parent                     No                                                  0.046369   
                              Prefer not to say                                   0.543610   
                              Yes                                                -0.020771   
lgbtq_status                  Bisexual                                           -0.088845   
                              Heterosexual                                        0.006043   
                              Homosexual                                          0.000859   
                              Other                                              -0.417898   
                              Prefer not to say                                   0.126560   
                              NaN                                                 0.030220   
personally_been_target        False                                               0.004599   
                              True                                               -0.017791   
personally_seen_toxic_content False                                              -0.056270   
                              True                                                0.023813   
political_affilation          Conservative                                       -0.017098   
                              Independent                                         0.009382   
                              Liberal                                             0.023707   
                              Other                                              -0.001496   
                              Prefer not to say                                  -0.033249   
religion_important            Not important    

In [ ]:
tasks.run_helper.run_result(
    df,
    sdb_column="random",
    value_col="toxic_score",
    comment_key_col="comment",
)